Created on Fri Jan 17 17:03:35 2019  
Group 9  
@authors:  

# Make the transition matrix from the database of events

In [1]:
import pandas as pd
import numpy as np
import json
from gensim.models import KeyedVectors

In [2]:
%run constants.ipynb
%run dictionnaries.ipynb
%run functions.ipynb

In [3]:
model_w2v = KeyedVectors.load_word2vec_format(pathword2vec, binary=True)

##### make the database

In [4]:
if True:
    prevoc = get_prevoc()
    voc = get_voc(model_w2v, prevoc)
    bdd = make_bdd(prevoc, 50)

bdd[CT_bdd_hash] = bdd[CT_bdd_json].apply(json_string_to_hash)
bdd.tail()

,session,date,json,hash
240,48,10/03/2014 06h01m23s,"{""tab"": ""Seat"", ""filters"": {""manufacturer"": [""...","{""filters"": {""aircraft"": [""il_87""], ""category""..."
241,48,22/10/2015 10h19m09s,"{""tab"": ""Plane plan"", ""filters"": {""manufacture...","{""filters"": {""aircraft"": [""177rg_cardinal_rg"",..."
242,48,00/06/2019 16h11m23s,"{""tab"": ""General"", ""filters"": {""manufacturer"":...","{""filters"": {""aircraft"": [""st_a"", ""traveler"", ..."
243,49,08/10/2018 16h34m10s,"{""tab"": ""Comments"", ""filters"": {""manufacturer""...","{""filters"": {""aircraft"": [""gulfstream_g100"", ""..."
244,49,21/11/2017 14h07m34s,"{""tab"": ""Plane plan"", ""filters"": {""manufacture...","{""filters"": {""aircraft"": [""bd_100_challenger_3..."


##### or load an existing one

In [5]:
bdd = pd.read_csv(bdd_directory+'df_bdd.csv', sep=',', engine='python',
                  index_col=0, encoding='utf-8').reset_index()

bdd[CT_bdd_hash] = bdd[CT_bdd_json].apply(json_string_to_hash)
bdd.tail()

,session,date,json,hash
5,0,07/11/2016 00h25m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""...","{""filters"": {""aircraft"": [""a350-900""], ""catego..."
6,0,07/11/2016 00h28m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""...","{""filters"": {""aircraft"": [""a320""], ""category"":..."
7,0,07/11/2016 00h30m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""...","{""filters"": {""aircraft"": [""a320""], ""category"":..."
8,0,07/11/2016 00h35m36s,"{""session"": 0, ""filters"": {""manufacturer"": [ ""...","{""filters"": {""aircraft"": [""a320""], ""category"":..."
9,0,07/11/2016 00h40m36s,"{""session"": 0, ""filters"": {""manufacturer"": [""A...","{""filters"": {""aircraft"": [], ""category"": [], ""..."


##### extract the states

In [6]:
etats = dict([(a, b)
              for (b, a) in enumerate(sorted(bdd[CT_bdd_hash].unique()))])
id_to_etats = dict((b, a) for (a, b) in etats.items())

##### extract the transitions

In [7]:
transitions = np.zeros([len(etats), len(etats)])
for session in bdd["session"].unique():
    for i in bdd[bdd["session"] == session].index[1:]:
        transitions[etats[bdd[CT_bdd_hash][i-1]],
                    etats[bdd[CT_bdd_hash][i]]] += 1

# to proba matrix
transitions = (transitions.T / np.sum(transitions, axis=1)).T
df_transitions = pd.DataFrame(transitions, columns=etats, index=etats)

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


##### make a prediction

In [8]:
a = id_to_etats[0]

predict_next_state(a, df_transitions)

'{"filters": {"aircraft": ["a350-900"], "category": [], "company": [], "country": ["england", "france", "germany", "italy", "spain"], "date": ["01092017", "30092017"], "manufacturer": ["airbus"]}, "tab": "general"}'

In [9]:
df_transitions.to_csv(bdd_directory+'df_transitions.csv',
                      sep='§', encoding='utf-8')